# Geometry Definition using OpenCASCADE (OCC) and GMSH

<b>Goals:</b>

We wish to generate a generate a geometry with smooth curves to avoid observed nuissances in the boundary layer mesh generation. We thus wish to understand how to apply the OpenCascade function <i>occ.fillet2D()</i>. This function is explained at [OpenCascade Modeling Algorithms](https://old.opencascade.com/doc/occt-6.7.0/overview/html/user_guides__modeling_algos.html); 

We wish to export the STL file per patch. To do so, go to "File -> Export" (very bottom of the File menu) and select .stl format. When prompted, select "Per physical surface".

<b>Notes:</b>
1. This version uses Julia as scripting language. It might be easier using Python. 

## Import Packages

In [1]:
try
    using Gmsh: Gmsh, gmsh
catch
    using gmsh
end 

using Plots

## Section 1: Introduction 

In [4]:
# this shows how to consult documentation 
#?gmsh.model.occ.addRectangle

## Section 2: Tutorial T19 (Original Version) Uses the Function Fillet() 

This [GMSH t19.jl](https://gitlab.onelab.info/gmsh/gmsh/blob/gmsh_4_15_0/tutorials/julia/t19.jl).
1. L43: why is <i>model.getBoundary</i> called twice? 

In [5]:
should_finalize = Gmsh.initialize()

gmsh.model.add("t19")

# Volumes can be constructed from (closed) curve loops thanks to the
# `addThruSections()' function
gmsh.model.occ.addCircle(0, 0, 0, 0.5, 1)
gmsh.model.occ.addCurveLoop([1], 1)
gmsh.model.occ.addCircle(0.1, 0.05, 1, 0.1, 2)
gmsh.model.occ.addCurveLoop([2], 2)
gmsh.model.occ.addCircle(-0.1, -0.1, 2, 0.3, 3)
gmsh.model.occ.addCurveLoop([3], 3)
gmsh.model.occ.addThruSections([1, 2, 3], 1)
gmsh.model.occ.synchronize()

# We can also force the creation of ruled surfaces:
gmsh.model.occ.addCircle(2 + 0, 0, 0, 0.5, 11)
gmsh.model.occ.addCurveLoop([11], 11)
gmsh.model.occ.addCircle(2 + 0.1, 0.05, 1, 0.1, 12)
gmsh.model.occ.addCurveLoop([12], 12)
gmsh.model.occ.addCircle(2 - 0.1, -0.1, 2, 0.3, 13)
gmsh.model.occ.addCurveLoop([13], 13)
gmsh.model.occ.addThruSections([11, 12, 13], 11, true, true)
gmsh.model.occ.synchronize()

# We copy the first volume, and fillet all its edges:
out = gmsh.model.occ.copy([(3, 1)])
gmsh.model.occ.translate(out, 4, 0, 0)
gmsh.model.occ.synchronize()
e = gmsh.model.getBoundary(gmsh.model.getBoundary(out), false)
gmsh.model.occ.fillet([out[1][2]], [abs(i[2]) for i in e], [0.1])
gmsh.model.occ.synchronize()

# OpenCASCADE also allows general extrusions along a smooth path. Let's first
# define a spline curve:
nturns = 1.
npts = 20
r = 1.
h = 1. * nturns
p = []
for i in 0:npts-1
    theta = i * 2 * pi * nturns / npts
    gmsh.model.occ.addPoint(r * cos(theta), r * sin(theta),
                            i * h / npts, 1, 1000 + i)
    push!(p, 1000 + i)
end
gmsh.model.occ.addSpline(p, 1000)

# A wire is like a curve loop, but open:
gmsh.model.occ.addWire([1000], 1000)

# We define the shape we would like to extrude along the spline (a disk):
gmsh.model.occ.addDisk(1, 0, 0, 0.2, 0.2, 1000)
gmsh.model.occ.rotate([(2, 1000)], 0, 0, 0, 1, 0, 0, pi / 2)

# We extrude the disk along the spline to create a pipe (other sweeping types
# can be specified; try e.g. "Frenet" instead of "DiscreteTrihedron"):
gmsh.model.occ.addPipe([(2, 1000)], 1000, "DiscreteTrihedron")

# We delete the source surface, and increase the number of sub-edges for a
# nicer display of the geometry:
gmsh.model.occ.remove([(2, 1000)])
gmsh.option.setNumber("Geometry.NumSubEdges", 1000)

gmsh.model.occ.synchronize()

# We can activate the calculation of mesh element sizes based on curvature
# (here with a target of 20 elements per 2*Pi radians):
gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 20)

# We can constraint the min and max element sizes to stay within reasonnable
# values (see `t10.jl' for more details):
gmsh.option.setNumber("Mesh.MeshSizeMin", 0.001)
gmsh.option.setNumber("Mesh.MeshSizeMax", 0.3)

gmsh.model.mesh.generate(3)
gmsh.write("t19.msh")

# Launch the GUI to see the results:
#if (true) gmsh.fltk.run() end 

gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 10%] Meshing curve 2 (Circle)
Info    : [ 10%] Meshing curve 3 (Circle)
Info    : [ 20%] Meshing curve 4 (BSpline)
Info    : [ 20%] Meshing curve 5 (BSpline)
Info    : [ 30%] Meshing curve 6 (BSpline)
Info    : [ 30%] Meshing curve 11 (Circle)
Info    : [ 40%] Meshing curve 12 (Circle)
Info    : [ 40%] Meshing curve 13 (Circle)
Info    : [ 40%] Meshing curve 14 (BSpline)
Info    : [ 50%] Meshing curve 15 (BSpline)
Info    : [ 50%] Meshing curve 16 (BSpline)
Info    : [ 60%] Meshing curve 17 (BSpline)
Info    : [ 60%] Meshing curve 18 (BSpline)
Info    : [ 70%] Meshing curve 19 (BSpline)
Info    : [ 70%] Meshing curve 20 (BSpline)
Info    : [ 70%] Meshing curve 21 (BSpline)
Info    : [ 80%] Meshing curve 22 (BSpline)
Info    : [ 80%] Meshing curve 1000 (BSpline)
Info    : [ 90%] Meshing curve 1001 (Ellipse)
Info    : [ 90%] Meshing curve 1002 (BSpline)
Info    : [100%] Meshing curve 1003 (BSpline)
Info    : [1

## Section 3: Unit square with one fillet extruded in the z-direction 

In [8]:
?gmsh.model.occ.addLine

```
gmsh.model.occ.addLine(startTag, endTag, tag = -1)
```

Add a straight line segment in the OpenCASCADE CAD representation, between the two points with tags `startTag` and `endTag`. If `tag` is positive, set the tag explicitly; otherwise a new tag is selected automatically. Return the tag of the line.

Return an integer.

Types:

  * `startTag`: integer
  * `endTag`: integer
  * `tag`: integer


In [18]:
#..initialize gmsh 
should_finalize = Gmsh.initialize()

#..generate geometry 
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.add("square")
#..set mesh density parameter 
lc = .1
zc = 0. 
#..define four points p1, p2, p3 and p4 with tags 1, 2, 3 and 4 via (x,y,z) coordinates 
p1 = gmsh.model.occ.addPoint(0, 0, zc, lc, 1)
p2 = gmsh.model.occ.addPoint(1., 0,  zc, lc, 2)
p3 = gmsh.model.occ.addPoint(1., 1., zc, lc, 3)
p4 = gmsh.model.occ.addPoint(0, 1., zc, lc, 4)
#..define four edges l1, l2, l3 and l4 with tags 1, 2, 3 and 4 by connecting point labels pairwise  
#..the line that connects points with tag i (first argument) and tag i+1 (second argument) has tag i 
l1 = gmsh.model.occ.addLine(1, 2, 1)
l2 = gmsh.model.occ.addLine(2, 3, 2)
l3 = gmsh.model.occ.addLine(3, 4, 3)
l4 = gmsh.model.occ.addLine(4, 1, 4)
#..define a fillet between lines with tag 1 and tag 2 and radius 0.1   
fillet_tag1 = gmsh.model.occ.fillet2D(1,2,0.1)
#..define curved loop by connecting four lines and fillet labels  
loop = gmsh.model.occ.addCurveLoop([1,fillet_tag1, 2, 3, 4], 1)
#..define surface by curved loop 
surf = gmsh.model.occ.addPlaneSurface([1], 1)
#..define volume by extracting the surface in perpendicular direction 
volume = gmsh.model.occ.extrude([(2, surf)], 0, 0, 1)

#..synchronize the CAD model 
gmsh.model.occ.synchronize()

if (true) gmsh.fltk.run() end

#..finalize gmsh 
should_finalize && Gmsh.finalize()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : MacOSX-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------
Info    : Writing '/Users/dlahaye/my_openfoam/milling/provs'...
Info    : Done writing '/Users/dlahaye/my_openfoam/milling/provs'
Info    : Writing '/Use

2026-02-04 19:17:13.432 julia[60317:5049829] The class 'NSSavePanel' overrides the method identifier.  This method is implemented by class 'NSWindow'


false

## Section 4: Geometry of mill using 10 points 

In [ ]:
#..initialize gmsh 
should_finalize = Gmsh.initialize()

#..generate geometry 
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.add("square")
#..set mesh density parameter 
lc = .1
zc = 0. 
#..define four points p1, p2, p3 and p4 with tags 1, 2, 3 and 4 via (x,y,z) coordinates 
p1 = gmsh.model.occ.addPoint(0, -0.1, zc, lc, 1)
p2 = gmsh.model.occ.addPoint(2., -0.1,  zc, lc, 2)
p3 = gmsh.model.occ.addPoint(2.5, -0.1, zc, lc, 3)
p4 = gmsh.model.occ.addPoint(2.5, 0., zc, lc, 4)
p5 = gmsh.model.occ.addPoint(2., 0., zc, lc, 5)
p6 = gmsh.model.occ.addPoint(1., 0., zc, lc, 6)
p7 = gmsh.model.occ.addPoint(0.5, 0.4, zc, lc, 7)
p8 = gmsh.model.occ.addPoint(0.25, 1., zc, lc, 8)
p9 = gmsh.model.occ.addPoint(0.25, 2.,  zc, lc, 9)
p10 = gmsh.model.occ.addPoint(0., 2.,  zc, lc, 10)

#..define four edges l1, l2, l3 and l4 with tags 1, 2, 3 and 4 by connecting point labels pairwise  
#..the line that connects points with tag i (first argument) and tag i+1 (second argument) has tag i 
l1 = gmsh.model.occ.addLine(1, 2, 1)
l2 = gmsh.model.occ.addLine(2, 3, 2)
l3 = gmsh.model.occ.addLine(3, 4, 3)
l4 = gmsh.model.occ.addLine(4, 5, 4)
l5 = gmsh.model.occ.addLine(5, 6, 5)
l6 = gmsh.model.occ.addLine(6, 7, 6)
l7 = gmsh.model.occ.addLine(7, 8, 7)
l8 = gmsh.model.occ.addLine(8, 9, 8)
l9 = gmsh.model.occ.addLine(9, 10, 9)
l10 = gmsh.model.occ.addLine(10, 1, 10)
#..define a fillet between lines with tag 1 and tag 2 and radius 0.1 
fillet_tag1 = gmsh.model.occ.fillet2D(5,6,0.5)
fillet_tag2 = gmsh.model.occ.fillet2D(6,7,0.5)
fillet_tag3 = gmsh.model.occ.fillet2D(7,8,0.5)
#..define curved loop by connecting four lines and fillet labels  
loop = gmsh.model.occ.addCurveLoop([1, 2, 3, 4, 5, fillet_tag1, 6, fillet_tag2, 7, fillet_tag3, 8, 9, 10], 12)
#..define surface by curved loop 
surf = gmsh.model.occ.addPlaneSurface([12], 1)
#..define volume by extracting the surface in perpendicular direction 
volume = gmsh.model.occ.extrude([(2, surf)], 0, 0, 1)

#..synchronize the CAD model 
gmsh.model.occ.synchronize()

if (true) gmsh.fltk.run() end

#..finalize gmsh 
should_finalize && Gmsh.finalize()